# Import Necessary Function and Library

In [1]:
import minsearch
import json
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [2]:
index = minsearch.Index(
    text_fields = {'question','text','section'},
    keyword_fields=['course']
)
index.fit(documents)
boost = {'question':3.0, 'section':0.5}

In [3]:
def search(query):
    boost = {'question':3.0, 'section':0.5}
    search_results = index.search(
        query=query,
        boost_dict = boost,
        filter_dict = {'couse':'data-engineering-zoomcamp'},
        num_results=5
    )
    return search_results
def build_prompt(query,search_results):
    prompt_template = """
    You are Course Assistant. You answer the question based on context. Use only fact the CONTEXT for answering the question.
    
    QUESTION: {question}
    CONTEXT : 
    {context}
    """.strip()
    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion:{doc['question']}\nanswer: {doc['text']}\n\n"
    prompt = prompt_template.format(question=query,context=search_results).strip()
    return prompt
def llm_open_api(prompt):
    client = OpenAI(api_key=api_key)
    response = client.chat.completions.create(model='gpt-3.5-turbo-16k',messages = [{'role':'user',"content":prompt}])
    return response.choices[0].message.content

In [4]:
from openai import OpenAI
import sys,os,os.path

In [6]:
api_key = os.environ['OPEN_AI_API_KEY']
client = OpenAI(api_key=api_key)

In [8]:
def llm_open_api(prompt):
    client = OpenAI(api_key=api_key)
    response = client.chat.completions.create(model='gpt-3.5-turbo-16k',messages = [{'role':'user',"content":prompt}])
    return response.choices[0].message.content

In [9]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query,search_results)
    answer = llm_open_api(prompt)
    return answer

In [10]:
rag('the course already started. Can I still enroll the course?')

"Yes, you can still enroll in the course. Although you won't be able to submit some of the homeworks, you can still take part in the course. If you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate."

# Use Ollama to Run Your LLM

In [11]:
from openai import OpenAI

In [12]:
def llm_ollama(prompt):
    client = OpenAI(
    base_url = 'http://localhost:11434/v1/',api_key = 'ollama',)
    response = client.chat.completions.create(model='phi3',messages = [{'role':'user',"content":prompt}])
    return response.choices[0].message.content

In [15]:
def rag_ollama(query):
    search_results = search(query)
    prompt = build_prompt(query,search_results)
    answer = llm_ollama(prompt)
    return answer

In [18]:
rag_ollama('the course already started. Can I still enroll the course?')

'Even though one of the provided contexts confirms that you won\'t be able to join after midstream: "Yes, but remember that there will already likely have been some work done and we are not starting at zero." It is essential for prospective students like yourself who wish to enroll later in life. Please communicate this clarification directly with your professors beforehand as it may vary by case or instructor\'s discretion, despite the information provided above.\n'

# Search with ElasticSearch

In [19]:
from elasticsearch import Elasticsearch

In [23]:
es_client = Elasticsearch('http://127.0.0.1:9200')

In [24]:
es_client.info()

ObjectApiResponse({'name': '1aa24fb22337', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'TPqLdMO1TXWTLi6mDEOwOw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [25]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = 'course_question'
es_client.indices.create(index=index_name,body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_question'})

In [26]:
from tqdm.auto import tqdm

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
for doc in documents:
    es_client.index(index=index_name,document=doc)

In [28]:
def elastic_query(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name,body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [29]:
def rag_elastic_ollama(query):
    search_results = elastic_query(query)
    prompt = build_prompt(query,search_results)
    answer = llm_ollama(prompt)
    return answer

In [32]:
query = 'the course already started. Can I still enroll the course?'
search_results = elastic_query(query)
print(search_results)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}, {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.', 'section': 'General course-related questions', 'question': 'Course - What can I do before the course starts?', 'course': 'data-engineering-zoomcamp'}, {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks

In [33]:
query = 'the course already started. Can I still enroll the course?'
rag_elastic_ollama(query)

KeyboardInterrupt: 